# Imports

In [2]:
import sys
import json
import pickle
import random
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout

Using TensorFlow backend.


# Load pickle file

In [3]:
'''Load pickle file of the string of lyrics'''

text = pickle.load(open("lyrics_text.p", "rb"))

In [3]:
'''Creates a set of each unique character found in the string'''

chars = set(text)

In [4]:
'''Prints the total number of characters and creates dictionaries of character indicies'''

print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 40


In [5]:
'''Dumps the character indicies variables since they are needed to write the song'''

pickle.dump(char_indices, open("char_indic.json", "wb"))
pickle.dump(indices_char, open("indic_char.json", "wb"))

# Neural Net

In [ ]:
'''Creates sentences out of the string it was fed and prints the number of sentences there are'''

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

In [ ]:
'''Vectorizes data'''

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
'''Builds model and complies using crossentropy'''

print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
'''Helper function to sample an index from a probability array'''

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
'''Where the neural net is actually trained and weights saved to be used to actually write the song in another notebook'''

for iteration in range(1, 60): #change back to 60
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, epochs=1)

    # dump the model to disk
    json_string = model.to_json()
    open("string.json", "w").write(json_string)
    model.save_weights("weights.h5", overwrite=False)
    
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [1.0]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()